In [ ]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

## 处理数据集

In [ ]:
# 将JSON文件转换为CSV文件
df = pd.read_json('/root/autodl-tmp/dataset/huanhuan.json')
ds = Dataset.from_pandas(df)
ds[:3]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('/root/autodl-tmp/models/glm-4-9b-chat', use_fast=False, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.eos_token

In [ ]:
def process_func(example):
    MAX_LENGTH = 384
    input_ids, attention_mask, labels = [], [], []
    # instruction = tokenizer((f"[gMASK]<sop><|system|>\n假设你是皇帝身边的女人--甄嬛。<|user|>\n"
    #                         f"{example['instruction']+example['input']}<|assistant|>\n"
    #                         ).strip(), 
    instruction = tokenizer((f"<|user|>\n{example['instruction']+example['input']}"
                            f"<|system|>\n假设你是皇帝身边的女人--甄嬛。<|assistant|>\n[gMASK]<sop>"
                            ).strip(), 
                            add_special_tokens=False)
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[-MAX_LENGTH:]
        attention_mask = attention_mask[-MAX_LENGTH:]
        labels = labels[-MAX_LENGTH:]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [ ]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

In [ ]:
# print(tokenizer.decode(tokenized_id[0]['input_ids']))

# print(tokenized_id[0]['input_ids'])

# print(tokenizer.decode([151331, 151333, 151335]))

# print(tokenizer.encode('[gMASK]<sop><|system|>', add_special_tokens=False))

In [ ]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[0]["labels"])))

In [ ]:
import torch

model = AutoModelForCausalLM.from_pretrained('/root/autodl-tmp/models/glm-4-9b-chat', device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True)
model

In [ ]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法
model.dtype


## LORA

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["query_key_value", "dense", "dense_h_to_4h", "dense_4h_to_h"],  # 现存问题只微调部分演示即可
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

In [ ]:
model = get_peft_model(model, config)
config

In [ ]:
model.print_trainable_parameters()


## 配置参数

In [ ]:
args = TrainingArguments(
    output_dir="/root/autodl-tmp/models/output/GLM4_post",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    logging_steps=25,
    num_train_epochs=2,
    save_steps=100,
    learning_rate=1e-5,
    save_on_each_node=True,
    gradient_checkpointing=True
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

In [ ]:
trainer.train()

## 保存lora和tokenizer结果

In [ ]:
peft_model_id="/root/autodl-tmp/lora/GLM4_post_lora"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)

In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import torch
# from peft import PeftModel

# mode_path = '/root/autodl-tmp/models/glm-4-9b-chat'
# lora_path = '/root/autodl-tmp/lora/GLM4_lora'

# # 加载tokenizer
# tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# # 加载模型
# model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# # 加载lora权重
# model = PeftModel.from_pretrained(model, model_id=lora_path)

# prompt = "你是谁？"
# inputs = tokenizer.apply_chat_template([{"role": "user", "content": "假设你是皇帝身边的女人--甄嬛。"},{"role": "user", "content": prompt}],
#                                        add_generation_prompt=True,
#                                        tokenize=True,
#                                        return_tensors="pt",
#                                        return_dict=True
#                                        ).to('cuda')


# gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
# with torch.no_grad():
#     outputs = model.generate(**inputs, **gen_kwargs)
#     outputs = outputs[:, inputs['input_ids'].shape[1]:]
#     print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = '/root/autodl-tmp/models/glm-4-9b-chat'
lora_path = '/root/autodl-tmp/lora/GLM4_post_lora'

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="cuda",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path).to("cuda")



/root/miniconda3/envs/dl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 10/10 [00:04<00:00,  2.08it/s]


In [ ]:
prompt = "结合你的身份，简单介绍下宝马i3,i4,i5，还有奔驰的一些，并给出理由，购买者为律师，价位在30w-50w之间，希望款式要新，动力强劲，必须彰显我的身份，"
# prompt = "结合你的身份，可以选择电车或者油车，宝马，具体情况如下：1、一定要最新的车型；2、购买者为律师，考虑商务，同时希望动力强劲；3、希望价格在30-40w之间"

inputs = tokenizer.apply_chat_template([{"role": "system", "content": "假设你是皇帝身边的女人--甄嬛。"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')

gen_kwargs = {"max_length": 512, "do_sample": True, "top_p": 0.8, "top_k": 3}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))


臣妾遵旨。宝马i3、i4、i5都是宝马旗下的纯电动车型，其中i3定位入门级，i4和i5则定位中高端。这三款车型都拥有时尚的外观、优秀的续航能力和强劲的动力。

对于律师这位购买者来说，我建议选择以下几款奔驰车型：

1. 奔驰E级插电混动版：售价约40-50万元，动力强劲，续航里程长，同时具备商务感和豪华感。这款车型的外观设计大气，内饰豪华，非常适合律师这样的专业人士。

2. 奔驰C级插电混动版：售价约30-40万元，动力表现优秀，续航里程适中，适合日常通勤和商务出行。C级车型外观时尚，内饰精致，彰显身份。

3. 奔驰GLC插电混动版：售价约40-50万元，是一款中型SUV，空间宽敞，动力强劲，适合家庭出行和商务接待。GLC的外观设计大气，内饰豪华，符合律师的身份。

理由如下：

1. 动力强劲：以上车型都采用了插电混动技术，动力表现优秀，满足驾驶需求。

2. 豪华感：奔驰品牌本身就是豪华品牌的代表，以上车型都具备较高的豪华感，符合律师的身份。

3. 续航里程：以上车型都具备一定的续航里程，满足日常通勤和商务出行的需求。

4. 外观设计：以上车型都拥有时尚、大气的造型，彰显个性。

综上所述，我建议律师购买奔驰E级插电混动版、奔驰C级插电混动版或奔驰GLC插电混动版。具体选择可根据个人喜好和需求来决定。


In [ ]:
# prompt = "介绍下tesla？"

# inputs = tokenizer.apply_chat_template([{"role": "user", "content": prompt},{"role": "system", "content": "假设你是皇帝身边的女人--甄嬛。"}],
#                                        add_generation_prompt=True,
#                                        tokenize=True,
#                                        return_tensors="pt",
#                                        return_dict=True
#                                        ).to('cuda')


# # gen_kwargs = {"max_length": 384, "do_sample": True, "top_k": 3, "temperature": 0.9}
# # gen_kwargs = {"max_length": 384, "do_sample": True, "top_p": 0.8}
# gen_kwargs = {"max_length": 384, "do_sample": True, "top_p": 0.8, "top_k": 3}
# with torch.no_grad():
#     outputs = model.generate(**inputs, **gen_kwargs)
#     outputs = outputs[:, inputs['input_ids'].shape[1]:]
#     print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### 推理的过程中测试出来的结果：
##### 1、数据量4000左右(1个epoch）可以达到语气修改的效果，但是仍然不够，需要更大量的数据（否则非常容易复读机）；
##### 2、增大topk，topp或者两个结合有效（例如topk=3/5，topp=0.8/0.9)；
##### 3、保证训练和测试的max_token一致，虽然roe可以推广到大长度，但是训练集如果只有短文本，后面会出现复读机；
##### 4、增加temperature无效；
##### 5、在prompt中限制输出的长度；